# Preprocessing the data

### Loading required packages

In [240]:
import pandas as pd
import numpy as np
import boto3
from datetime import datetime
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import StandardScaler
import locale
from sklearn.model_selection import train_test_split

## Meteo data

### Load the meteo data

In [671]:
# Loading meteo data
quarters = ['Q1', 'Q2', 'Q3', 'Q4']
years = ['2022']
base_url_meteo = 'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Meteo+data/LC_{}{}.csv'

dfs = []

for year in years:
    for quarter in quarters:
        url = base_url_meteo.format(year, quarter)
        df = pd.read_csv(url)
        dfs.append(df)

### Build pipeline

In [674]:
## Building preprocessing pipeline
# Step 1: Concatenate datasets
def concatenate_datasets(dfs):
    return pd.concat(dfs, ignore_index=True)

# Step 2: Convert UTC to CEST by adding 2 hours
def convert_utc_to_cest(df):
    df['DATEUTC'] = pd.to_datetime(df['DATEUTC'])
    df['DATECEST'] = df['DATEUTC']+ pd.Timedelta(hours=2)
    return df

# Steo 3: Update the month day hour columns to CEST
def convert_time(df):
    df['Month'] = df['DATECEST'].dt.month
    df['Day'] = df['DATECEST'].dt.day
    df['Hour'] = df['DATECEST'].dt.hour
    return df

# Step 4: Drop columns
def drop_columns(df):
    columns_to_keep = ['DATECEST', 'LC_RAININ', 'LC_DAILYRAIN', 'LC_WINDDIR', 'LC_WINDSPEED', 'LC_TEMP_QCL3', 'Month', 'Day', 'Hour']  #there's less columns we keep than drop
    columns_to_drop = set(df.columns) - set(columns_to_keep)
    return df.drop(columns=columns_to_drop)

# Step 5: Check for percentage of missing values in each column
def print_null_percentage(df):
    null_percentage = df.isnull().sum() / len(df)
    print('The percentage of missing values in each column')
    print(null_percentage)
    return df

# Step 6: Forward fill missing values
def forward_fill(df):
    return df.ffill()

# Step 7: Check whether there are missing values left
def check_missing_values(df):
    missing_values = df.isnull().sum()
    print('Check whether there are missing values left')
    print(missing_values)
    return df

# Step 8: Calculate summary statistics for daily rain sum
def daily_rain_sum(df):
    summary_stats = df['LC_DAILYRAIN'].describe()
    print('Summary statistics for daily rain sum')
    print(summary_stats)
    return df

# Step 9: Calculate fraction of non-zero values in the 'LC_DAILYRAIN' column
def non_zero_fraction(df):
    nonzero_count = np.count_nonzero(df['LC_DAILYRAIN'])
    non_zero_frac = nonzero_count/len(df)
    print("Fraction of non-zero values:", non_zero_frac)
    return df

# Define the pipeline
pipeline_meteo = Pipeline([
    ('concatenate_datasets', FunctionTransformer(concatenate_datasets)),
    ('convert_utc_to_cest', FunctionTransformer(convert_utc_to_cest)),
    ('convert_time', FunctionTransformer(convert_time)),
    ('drop_columns', FunctionTransformer(drop_columns)),
    ('print_null_percentage', FunctionTransformer(print_null_percentage)),
    ('forward_fill', FunctionTransformer(forward_fill)),
    ('check_missing_values', FunctionTransformer(check_missing_values)),
    ('daily_rain_sum', FunctionTransformer(daily_rain_sum)),
    ('non_zero_fraction', FunctionTransformer(non_zero_fraction))
])

### Apply the pipeline and generate hourly, daily, and monthly meteo data

In [675]:
# Apply the pipeline
meteo_combined_df = pipeline_meteo.fit_transform(dfs)
meteo_combined_df.head()

The percentage of missing values in each column
LC_RAININ       0.056770
LC_DAILYRAIN    0.056770
LC_WINDDIR      0.056770
LC_WINDSPEED    0.056770
Month           0.000000
Day             0.000000
Hour            0.000000
LC_TEMP_QCL3    0.062285
DATECEST        0.000000
dtype: float64
Check whether there are missing values left
LC_RAININ       0
LC_DAILYRAIN    0
LC_WINDDIR      0
LC_WINDSPEED    0
Month           0
Day             0
Hour            0
LC_TEMP_QCL3    0
DATECEST        0
dtype: int64
Summary statistics for daily rain sum
count    5.546880e+06
mean     1.319783e-03
std      6.177559e-03
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      1.540000e-01
Name: LC_DAILYRAIN, dtype: float64
Fraction of non-zero values: 0.17391488548517364


,LC_RAININ,LC_DAILYRAIN,LC_WINDDIR,LC_WINDSPEED,Month,Day,Hour,LC_TEMP_QCL3,DATECEST
0,0.0,0.0,-169.0,0.43,1,1,2,13.048027,2022-01-01 02:10:00
1,0.0,0.0,-170.0,0.33,1,1,2,12.985849,2022-01-01 02:20:00
2,0.0,0.0,-167.0,0.46,1,1,2,12.950322,2022-01-01 02:30:00
3,0.0,0.0,-160.0,0.52,1,1,2,12.949550,2022-01-01 02:40:00
4,0.0,0.0,-166.0,0.51,1,1,2,12.952268,2022-01-01 02:50:00


In [676]:
# Create dataframe per hour

# Specify the aggregation function for each column
  # for LC_DAILYRAIN we take the last value because it's cumulative, for other columns the mean
aggregations = {
    'LC_DAILYRAIN': 'mean',  # Select the last value for 'LC_DAILYRAIN' ###TAKE MEAN FOR NOW TO MAKE THE GRAPHS LOOK OK
    'LC_RAININ': 'mean',  
    'LC_WINDDIR': 'mean',
    'LC_WINDDIR': 'mean', 
    'LC_WINDSPEED': 'mean', 
    'LC_TEMP_QCL3': 'mean'
}

# Perform the groupby aggregation
meteo_per_hour = meteo_combined_df.groupby(['Month', 'Day', 'Hour']).mean()
meteo_per_hour = meteo_per_hour.reset_index()
meteo_per_hour.head()

,Month,Day,Hour,LC_RAININ,LC_DAILYRAIN,LC_WINDDIR,LC_WINDSPEED,LC_TEMP_QCL3,DATECEST
0,1,1,0,0.000023,0.002997,-33.566358,1.487099,15.513391,2023-01-01 00:25:00.000000000
1,1,1,1,0.000019,0.002174,-29.188272,1.465571,15.770757,2023-01-01 01:25:00.000000000
2,1,1,2,0.000003,0.000360,-18.197324,0.389565,13.100358,2022-03-08 00:28:59.799331072
3,1,1,3,0.000007,0.000000,-16.227891,0.222602,12.669197,2022-01-01 03:25:00.000000000
4,1,1,4,0.000009,0.000000,-13.710884,0.217194,12.520271,2022-01-01 04:25:00.000000000


In [677]:
# Create dataframe per day

# still the same "aggregations" as before
meteo_per_day = meteo_combined_df.groupby(['Month', 'Day']).mean()
meteo_per_day = meteo_per_day.reset_index()
meteo_per_day.head()

,Month,Day,LC_RAININ,LC_DAILYRAIN,LC_WINDDIR,LC_WINDSPEED,Hour,LC_TEMP_QCL3,DATECEST
0,1,1,0.000004,0.000275,-7.446286,0.414860,11.400646,12.524393,2022-02-06 11:23:32.498244864
1,1,2,0.000654,0.002608,-25.975694,0.649436,11.500000,12.004777,2022-01-02 11:55:00.000000000
2,1,3,0.000675,0.006600,-37.386338,0.711017,11.500000,9.769569,2022-01-03 11:55:00.000000000
3,1,4,0.000506,0.003867,-23.273101,0.344787,11.500000,7.158320,2022-01-04 11:55:00.000000000
4,1,5,0.000089,0.000738,-44.453160,0.603273,11.500000,3.790048,2022-01-05 11:55:00.000000000


In [678]:
# Create dataframe per month

# still the same "aggregations" as before
meteo_per_month = meteo_combined_df.groupby(['Month']).mean()
meteo_per_month = meteo_per_month.reset_index()
meteo_per_month.head()

,Month,LC_RAININ,LC_DAILYRAIN,LC_WINDDIR,LC_WINDSPEED,Day,Hour,LC_TEMP_QCL3,DATECEST
0,1,0.000112,0.001034,-16.307700,0.339932,15.995544,11.496766,4.733596,2022-01-17 15:54:43.257389312
1,2,0.000131,0.001263,-25.317653,0.741510,14.500000,11.500000,6.929743,2022-02-14 23:54:59.999999744
2,3,0.000010,0.000104,12.215986,0.250748,16.000000,11.500000,8.108503,2022-03-16 11:55:00.000000000
3,4,0.000054,0.000504,3.631846,0.369361,15.504041,11.503034,10.690818,2022-04-16 00:01:00.124777728
4,5,0.000076,0.000653,-9.013140,0.240605,16.000000,11.500000,15.568973,2022-05-16 11:55:00.000000000


In [63]:
'''
# export dataframes (only needs to be ran once so comment it out)
meteo_per_hour.to_csv('hourly_weatherdata_2022.csv', index=False)
meteo_per_day.to_csv('daily_weatherdata_2022.csv', index=False)
meteo_per_month.to_csv('monthly_weatherdata_2022.csv', index=False)
'''

In [679]:
# Delete dfs to reduce memory use
del dfs
del meteo_combined_df
del meteo_per_hour
del meteo_per_day
del meteo_per_month

## Noise level data

### Loading datasets

- January 

In [4]:
# Define a list of URLs
urls_jan = [
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Jan/Jan/csv_results_42_255439_mp-01-naamsestraat-35-maxim.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Jan/Jan/csv_results_42_255440_mp-02-naamsestraat-57-xior.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Jan/Jan/csv_results_42_255441_mp-03-naamsestraat-62-taste.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Jan/Jan/csv_results_42_255442_mp-05-calvariekapel-ku-leuven.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Jan/Jan/csv_results_42_255443_mp-06-parkstraat-2-la-filosovia.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Jan/Jan/csv_results_42_255444_mp-07-naamsestraat-81.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Jan/Jan/csv_results_42_255445_mp-08-kiosk-stadspark.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Jan/Jan/csv_results_42_280324_mp08bis---vrijthof.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Jan/Jan/csv_results_42_303910_mp-04-his-hears.csv'
   ]

# Create an empty list to store the DataFrames
dfs_jan = []

# Loop through each URL and read the CSV into a DataFrame
for url_jan in urls_jan:
    df_jan = pd.read_csv(url_jan, header=0, sep=';')
    dfs_jan.append(df_jan)

# Now we have a list of DataFrames for each URL called dfs_jan

- February

In [5]:
# Define a list of URLs 
urls_feb = [
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Feb/Feb/csv_results_42_255439_mp-01-naamsestraat-35-maxim.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Feb/Feb/csv_results_42_255440_mp-02-naamsestraat-57-xior.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Feb/Feb/csv_results_42_255441_mp-03-naamsestraat-62-taste.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Feb/Feb/csv_results_42_255442_mp-05-calvariekapel-ku-leuven.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Feb/Feb/csv_results_42_255443_mp-06-parkstraat-2-la-filosovia.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Feb/Feb/csv_results_42_255444_mp-07-naamsestraat-81.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Feb/Feb/csv_results_42_255445_mp-08-kiosk-stadspark.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Feb/Feb/csv_results_42_280324_mp08bis---vrijthof.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Feb/Feb/csv_results_42_303910_mp-04-his-hears.csv'
   ]

# Create an empty list to store the DataFrames
dfs_feb = []

# Loop through each URL and read the CSV into a DataFrame
for url_feb in urls_feb:
    df_feb = pd.read_csv(url_feb, header=0, sep=';')
    dfs_feb.append(df_feb)

# Now we have a list of DataFrames for each URL called dfs_feb

- March

In [6]:
# Define a list of URLs 
urls_mar = [
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/March/March/csv_results_44_255439_mp-01-naamsestraat-35-maxim.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/March/March/csv_results_44_255440_mp-02-naamsestraat-57-xior.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/March/March/csv_results_44_255441_mp-03-naamsestraat-62-taste.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/March/March/csv_results_44_255442_mp-05-calvariekapel-ku-leuven.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/March/March/csv_results_44_255443_mp-06-parkstraat-2-la-filosovia.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/March/March/csv_results_44_255444_mp-07-naamsestraat-81.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/March/March/csv_results_44_255445_mp-08-kiosk-stadspark.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/March/March/csv_results_44_280324_mp08bis---vrijthof.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/March/March/csv_results_44_303910_mp-04-his-hears.csv'
   ]

# Create an empty list to store the DataFrames
dfs_mar = []

# Loop through each URL and read the CSV into a DataFrame
for url_mar in urls_mar:
    df_mar = pd.read_csv(url_mar, header=0, sep=';')
    dfs_mar.append(df_mar)

- April

In [7]:
# Define a list of URLs 
urls_apr = [
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/April/April/csv_results_45_255439_mp-01-naamsestraat-35-maxim.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/April/April/csv_results_45_255440_mp-02-naamsestraat-57-xior.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/April/April/csv_results_45_255441_mp-03-naamsestraat-62-taste.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/April/April/csv_results_45_255442_mp-05-calvariekapel-ku-leuven.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/April/April/csv_results_45_255443_mp-06-parkstraat-2-la-filosovia.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/April/April/csv_results_45_255444_mp-07-naamsestraat-81.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/April/April/csv_results_45_255445_mp-08-kiosk-stadspark.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/April/April/csv_results_45_280324_mp08bis---vrijthof.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/April/April/csv_results_45_303910_mp-04-his-hears.csv'
   ]

# Create an empty list to store the DataFrames
dfs_apr = []

# Loop through each URL and read the CSV into a DataFrame
for url_apr in urls_apr:
    df_apr = pd.read_csv(url_apr, header=0, sep=';')
    dfs_apr.append(df_apr)

- May

In [8]:
# Define a list of URLs 
urls_may = [
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/May/May/csv_results_46_255439_mp-01-naamsestraat-35-maxim.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/May/May/csv_results_46_255440_mp-02-naamsestraat-57-xior.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/May/May/csv_results_46_255441_mp-03-naamsestraat-62-taste.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/May/May/csv_results_46_255442_mp-05-calvariekapel-ku-leuven.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/May/May/csv_results_46_255443_mp-06-parkstraat-2-la-filosovia.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/May/May/csv_results_46_255444_mp-07-naamsestraat-81.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/May/May/csv_results_46_255445_mp-08-kiosk-stadspark.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/May/May/csv_results_46_280324_mp08bis---vrijthof.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/May/May/csv_results_46_303910_mp-04-his-hears.csv'
   ]

# Create an empty list to store the DataFrames
dfs_may = []

# Loop through each URL and read the CSV into a DataFrame
for url_may in urls_may:
    df_may = pd.read_csv(url_may, header=0, sep=';')
    dfs_may.append(df_may)


- June

In [9]:
# Define a list of URLs 
url_jun = [
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/June/June/csv_results_47_255439_mp-01-naamsestraat-35-maxim.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/June/June/csv_results_47_255440_mp-02-naamsestraat-57-xior.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/June/June/csv_results_47_255441_mp-03-naamsestraat-62-taste.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/June/June/csv_results_47_255442_mp-05-calvariekapel-ku-leuven.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/June/June/csv_results_47_255443_mp-06-parkstraat-2-la-filosovia.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/June/June/csv_results_47_255444_mp-07-naamsestraat-81.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/June/June/csv_results_47_255445_mp-08-kiosk-stadspark.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/June/June/csv_results_47_280324_mp08bis---vrijthof.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/June/June/csv_results_47_303910_mp-04-his-hears.csv'
   ]

# Create an empty list to store the DataFrames
dfs_jun = []

# Loop through each URL and read the CSV into a DataFrame
for url_jun in url_jun:
    df_jun = pd.read_csv(url_jun, header=0, sep=';')
    dfs_jun.append(df_jun)

- July

In [10]:
# Define a list of URLs 
urls_jul = [
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Jul/Jul/csv_results_48_255439_mp-01-naamsestraat-35-maxim.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Jul/Jul/csv_results_48_255440_mp-02-naamsestraat-57-xior.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Jul/Jul/csv_results_48_255441_mp-03-naamsestraat-62-taste.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Jul/Jul/csv_results_48_255442_mp-05-calvariekapel-ku-leuven.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Jul/Jul/csv_results_48_255443_mp-06-parkstraat-2-la-filosovia.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Jul/Jul/csv_results_48_255444_mp-07-naamsestraat-81.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Jul/Jul/csv_results_48_255445_mp-08-kiosk-stadspark.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Jul/Jul/csv_results_48_280324_mp08bis---vrijthof.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Jul/Jul/csv_results_48_303910_mp-04-his-hears.csv'
   ]

# Create an empty list to store the DataFrames
dfs_jul = []

# Loop through each URL and read the CSV into a DataFrame
for url_jul in urls_jul:
    df_jul = pd.read_csv(url_jul, header=0, sep=';')
    dfs_jul.append(df_jul)

- August

In [11]:
# Define a list of URLs 
urls_aug = [
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Aug/Aug/csv_results_49_255439_mp-01-naamsestraat-35-maxim.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Aug/Aug/csv_results_49_255440_mp-02-naamsestraat-57-xior.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Aug/Aug/csv_results_49_255441_mp-03-naamsestraat-62-taste.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Aug/Aug/csv_results_49_255442_mp-05-calvariekapel-ku-leuven.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Aug/Aug/csv_results_49_255443_mp-06-parkstraat-2-la-filosovia.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Aug/Aug/csv_results_49_255444_mp-07-naamsestraat-81.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Aug/Aug/csv_results_49_255445_mp-08-kiosk-stadspark.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Aug/Aug/csv_results_49_280324_mp08bis---vrijthof.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Aug/Aug/csv_results_49_303910_mp-04-his-hears.csv'
   ]

# Create an empty list to store the DataFrames
dfs_aug = []

# Loop through each URL and read the CSV into a DataFrame
for url_aug in urls_aug:
    df_aug = pd.read_csv(url_aug, header=0, sep=';')
    dfs_aug.append(df_aug)

- September

In [12]:
# Define a list of URLs 
urls_sep = [
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Sep/Sep/csv_results_50_255439_mp-01-naamsestraat-35-maxim.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Sep/Sep/csv_results_50_255440_mp-02-naamsestraat-57-xior.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Sep/Sep/csv_results_50_255441_mp-03-naamsestraat-62-taste.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Sep/Sep/csv_results_50_255442_mp-05-calvariekapel-ku-leuven.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Sep/Sep/csv_results_50_255443_mp-06-parkstraat-2-la-filosovia.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Sep/Sep/csv_results_50_255444_mp-07-naamsestraat-81.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Sep/Sep/csv_results_50_255445_mp-08-kiosk-stadspark.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Sep/Sep/csv_results_50_280324_mp08bis---vrijthof.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Sep/Sep/csv_results_50_303910_mp-04-his-hears.csv'
   ]

# Create an empty list to store the DataFrames
dfs_sep = []

# Loop through each URL and read the CSV into a DataFrame
for url_sep in urls_sep:
    df_sep = pd.read_csv(url_sep, header=0, sep=';')
    dfs_sep.append(df_sep)

- October

In [13]:
# Define a list of URLs 
urls_oct = [
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Oct/Oct/csv_results_51_255439_mp-01-naamsestraat-35-maxim.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Oct/Oct/csv_results_51_255440_mp-02-naamsestraat-57-xior.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Oct/Oct/csv_results_51_255441_mp-03-naamsestraat-62-taste.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Oct/Oct/csv_results_51_255442_mp-05-calvariekapel-ku-leuven.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Oct/Oct/csv_results_51_255443_mp-06-parkstraat-2-la-filosovia.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Oct/Oct/csv_results_51_255444_mp-07-naamsestraat-81.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Oct/Oct/csv_results_51_255445_mp-08-kiosk-stadspark.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Oct/Oct/csv_results_51_280324_mp08bis---vrijthof.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Oct/Oct/csv_results_51_303910_mp-04-his-hears.csv'
   ]

# Create an empty list to store the DataFrames
dfs_oct = []

# Loop through each URL and read the CSV into a DataFrame
for url_oct in urls_oct:
    df_oct = pd.read_csv(url_oct, header=0, sep=';')
    dfs_oct.append(df_oct)

- November

In [14]:
# Define a list of URLs 
urls_nov = [
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Nov/Nov/csv_results_52_255439_mp-01-naamsestraat-35-maxim.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Nov/Nov/csv_results_52_255440_mp-02-naamsestraat-57-xior.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Nov/Nov/csv_results_52_255441_mp-03-naamsestraat-62-taste.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Nov/Nov/csv_results_52_255442_mp-05-calvariekapel-ku-leuven.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Nov/Nov/csv_results_52_255443_mp-06-parkstraat-2-la-filosovia.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Nov/Nov/csv_results_52_255444_mp-07-naamsestraat-81.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Nov/Nov/csv_results_52_255445_mp-08-kiosk-stadspark.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Nov/Nov/csv_results_52_280324_mp08bis---vrijthof.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Nov/Nov/csv_results_52_303910_mp-04-his-hears.csv'
   ]

# Create an empty list to store the DataFrames
dfs_nov = []

# Loop through each URL and read the CSV into a DataFrame
for url_nov in urls_nov:
    df_nov = pd.read_csv(url_nov, header=0, sep=';')
    dfs_nov.append(df_nov)

- December

In [15]:
# Define a list of URLs 
urls_dec = [
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Dec/Dec/csv_results_53_255439_mp-01-naamsestraat-35-maxim.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Dec/Dec/csv_results_53_255440_mp-02-naamsestraat-57-xior.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Dec/Dec/csv_results_53_255441_mp-03-naamsestraat-62-taste.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Dec/Dec/csv_results_53_255442_mp-05-calvariekapel-ku-leuven.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Dec/Dec/csv_results_53_255443_mp-06-parkstraat-2-la-filosovia.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Dec/Dec/csv_results_53_255444_mp-07-naamsestraat-81.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Dec/Dec/csv_results_53_255445_mp-08-kiosk-stadspark.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Dec/Dec/csv_results_53_280324_mp08bis---vrijthof.csv',
    'https://mda-georgia-bucket.s3.eu-central-1.amazonaws.com/Noise+data/Dec/Dec/csv_results_53_303910_mp-04-his-hears.csv'
   ]

# Create an empty list to store the DataFrames
dfs_dec = []

# Loop through each URL and read the CSV into a DataFrame
for url_dec in urls_dec:
    df_dec = pd.read_csv(url_dec, header=0, sep=';')
    dfs_dec.append(df_dec)

In [119]:
# List of datasets
dfs_2022 = [dfs_jan,dfs_feb,dfs_mar,dfs_apr,dfs_may,dfs_jun,dfs_jul,dfs_aug,dfs_sep,dfs_oct,dfs_nov,dfs_dec]

### For visualization purpose

##### Build pipleines

- Overall dataset preprocessing pipeline

In [564]:
# Step 1: Concatenate datasets
def concatenate_datasets(dfs):
    return pd.concat(dfs, ignore_index=True)

# Step 2: Convert timestamps to datetime
def convert_to_datetime(df):
    df['result_timestamp'] = pd.to_datetime(df['result_timestamp'], format='%d/%m/%Y %H:%M:%S.%f')
    return df

# Step 3: Extract month, day, hour, minute from timestamps
def extract_time(df):
    df['month'] = df['result_timestamp'].dt.month
    df['day'] = df['result_timestamp'].dt.day
    df['hour'] = df['result_timestamp'].dt.hour
    #   df['minute'] = df['result_timestamp'].dt.minute
    return df

# Step 4: Drop columns
def drop_columns(df):
    columns_to_keep = ['description', 'lamax', 'laeq', 'month', 'day', 'hour'] #also minute if we calculate it
    columns_to_drop = set(df.columns) - set(columns_to_keep)
    return df.drop(columns=columns_to_drop)

# Step 5: Forward fill missing values
def forward_fill(df):
    return df.ffill()

# Define the pipeline
pipeline_general = Pipeline([
    ('concatenate_datasets', FunctionTransformer(concatenate_datasets)),
    ('convert_to_datetime', FunctionTransformer(convert_to_datetime)),
    ('extract_time', FunctionTransformer(extract_time)),
    ('drop_columns', FunctionTransformer(drop_columns)),
    ('forward_fill', FunctionTransformer(forward_fill))
])

- Conintue to get hourly, daily, and monthly aggregated data

In [565]:
## Hourly pipeline
# Step 6: Perform groupby to create dataframe per hour
def perform_groupby(df):
    return df.groupby(['month', 'day', 'hour', 'description']).mean()

# Step 7: Reset index
def reset_index_func(df):
    return df.reset_index()

'''
# Step 8: Standardize the data
def standardize_columns(df, columns_to_standardize):
    scaler = StandardScaler()
    standardized_values = scaler.fit_transform(df[columns_to_standardize])
    new_columns = [column + '_standardized' for column in columns_to_standardize]
    df[new_columns] = pd.DataFrame(standardized_values, columns=new_columns)
    return df
'''

# Step 9: Define a custom transformer to create the new column date
class DateTransformer:
    def transform(self, df):
        df['year'] = 2022
        df['date'] = df.apply(lambda row: pd.to_datetime(f"{int(row['day']):02d}-{int(row['month']):02d}-{int(row['year']):04d}-{int(row['hour']):02d}", format='%d-%m-%Y-%H'), axis=1)
        df['date'] = df['date'].dt.strftime('%H:%M %d-%m-%Y')
        return df

    def fit(self, df, y=None):
        return self
    
# Step 10: Drop the year column
def drop_year_column(df):
    return df.drop(columns='year')

# Define the pipeline
pipeline_hourly = Pipeline([
    ('groupby', FunctionTransformer(perform_groupby)),
    ('reset_index', FunctionTransformer(reset_index_func)),
    #('standardize_columns', FunctionTransformer(standardize_columns, kw_args={'columns_to_standardize': ['lamax', 'laeq']})),
    ('date_transformer', DateTransformer()),
    ('drop_year_column', FunctionTransformer(drop_year_column))
])

In [566]:
## Daily pipeline
# Step 6: Perform groupby to create dataframe per hour
def perform_groupby(df):
    return df.groupby(['month', 'day', 'description']).mean()

# Step 7: Reset index
def reset_index_func(df):
    return df.reset_index()

'''
# Step 8: Standardize the data
def standardize_columns(df, columns_to_standardize):
    scaler = StandardScaler()
    standardized_values = scaler.fit_transform(df[columns_to_standardize])
    new_columns = [column + '_standardized' for column in columns_to_standardize]
    df[new_columns] = pd.DataFrame(standardized_values, columns=new_columns)
    return df
'''

# Step 9: Drop unwanted columns
def drop_columns(df):
    return df.drop(columns='hour')

# Step 10: Define a custom transformer to create the new column date
class DateTransformer:
    def transform(self, df):
        df['year'] = 2022
        df['date'] = df.apply(lambda row: pd.to_datetime(f"{int(row['day']):02d}-{int(row['month']):02d}-{int(row['year']):04d}", format='%d-%m-%Y'), axis=1)
        df['date'] = df['date'].dt.strftime('%d-%m-%Y')
        return df

    def fit(self, df, y=None):
        return self
    
# Step 11: Drop the year column
def drop_year_column(df):
    return df.drop(columns='year')

# Define the pipeline
pipeline_daily = Pipeline([
    ('groupby', FunctionTransformer(perform_groupby)),
    ('reset_index', FunctionTransformer(reset_index_func)),
    #('standardize_columns', FunctionTransformer(standardize_columns, kw_args={'columns_to_standardize': ['lamax', 'laeq']})),
    ('drop_columns', FunctionTransformer(drop_columns)),
    ('date_transformer', DateTransformer()),
    ('drop_year_column', FunctionTransformer(drop_year_column))
])

In [567]:
## Monthly pipeline
# Step 6: Perform groupby to create dataframe per hour
def perform_groupby(df):
    return df.groupby(['month', 'description']).mean()

# Step 7: Reset index
def reset_index_func(df):
    return df.reset_index()

'''
# Step 8: Standardize the data
def standardize_columns(df, columns_to_standardize):
    scaler = StandardScaler()
    standardized_values = scaler.fit_transform(df[columns_to_standardize])
    new_columns = [column + '_standardized' for column in columns_to_standardize]
    df[new_columns] = pd.DataFrame(standardized_values, columns=new_columns)
    return df
'''

# Step 9: Drop unwanted columns
def drop_columns(df):
    columns_to_drop = ['day', 'hour']
    return df.drop(columns=columns_to_drop)

# Step 10: Define a custom transformer to create the new column date
class DateTransformer:
    def transform(self, df):
        df['year'] = 2022
        df['date'] = df.apply(lambda row: pd.to_datetime(f"{int(row['month']):02d}-{int(row['year']):04d}", format='%m-%Y'), axis=1)
        df['date'] = df['date'].dt.strftime('%b %Y')
        return df

    def fit(self, df, y=None):
        return self
    
# Step 11: Drop the year column
def drop_year_column(df):
    return df.drop(columns='year')

# Define the pipeline
pipeline_monthly = Pipeline([
    ('groupby', FunctionTransformer(perform_groupby)),
    ('reset_index', FunctionTransformer(reset_index_func)),
    #('standardize_columns', FunctionTransformer(standardize_columns, kw_args={'columns_to_standardize': ['lamax', 'laeq']})),
    ('drop_columns', FunctionTransformer(drop_columns)),
    ('date_transformer', DateTransformer()),
    ('drop_year_column', FunctionTransformer(drop_year_column))
])

##### Apply the pipelines

In [568]:
# Apply the pipeline to the overall dataset
transformed_overall_datasets = []
for df_2022 in dfs_2022:
    transformed_overall_dataset = pipeline_general.fit_transform(df_2022)
    transformed_overall_datasets.append(transformed_overall_dataset)

combined = pd.concat(transformed_overall_datasets, ignore_index=True)
combined.head(100000)

,description,lamax,laeq,month,day,hour
0,MP 03: Naamsestraat 62 Taste,87.6,82.7,1,1,0
1,MP 03: Naamsestraat 62 Taste,84.5,83.1,1,1,0
2,MP 03: Naamsestraat 62 Taste,84.8,82.7,1,1,0
3,MP 03: Naamsestraat 62 Taste,81.9,79.3,1,1,0
4,MP 03: Naamsestraat 62 Taste,78.3,76.0,1,1,0
...,...,...,...,...,...,...
99995,MP 03: Naamsestraat 62 Taste,45.9,44.7,1,2,3
99996,MP 03: Naamsestraat 62 Taste,46.2,45.4,1,2,3
99997,MP 03: Naamsestraat 62 Taste,46.0,44.6,1,2,3
99998,MP 03: Naamsestraat 62 Taste,45.5,44.7,1,2,3


In [569]:
# Apply the pipeline to the combined and collect hourly data
combined_hourly = pipeline_hourly.fit_transform(combined) # this df will also be used for modelling purpose later 

combined_hourly.head(100000)

,month,day,hour,description,lamax,laeq,date
0,1,1,0,MP 03: Naamsestraat 62 Taste,60.322528,57.126833,00:00 01-01-2022
1,1,1,0,MP 05: Calvariekapel KU Leuven,53.230972,49.987639,00:00 01-01-2022
2,1,1,0,MP 06: Parkstraat 2 La Filosovia,53.666056,50.752000,00:00 01-01-2022
3,1,1,0,MP 07: Naamsestraat 81,50.056861,47.440222,00:00 01-01-2022
4,1,1,1,MP 03: Naamsestraat 62 Taste,53.033583,50.853806,01:00 01-01-2022
...,...,...,...,...,...,...,...
55464,12,31,23,MP 04: His & Hears,61.517005,58.995888,23:00 31-12-2022
55465,12,31,23,MP 05: Calvariekapel KU Leuven,59.659572,56.671964,23:00 31-12-2022
55466,12,31,23,MP 06: Parkstraat 2 La Filosovia,57.888775,55.366713,23:00 31-12-2022
55467,12,31,23,MP 07: Naamsestraat 81,55.636732,53.113476,23:00 31-12-2022


In [570]:
# Apply the pipeline to the combined and collect daily data
combined_daily = pipeline_daily.fit_transform(combined) # this df will also be used for modelling purpose later

combined_daily.head(100000)

,month,day,description,lamax,laeq,date
0,1,1,MP 03: Naamsestraat 62 Taste,51.665242,49.992637,01-01-2022
1,1,1,MP 05: Calvariekapel KU Leuven,48.747476,46.504067,01-01-2022
2,1,1,MP 06: Parkstraat 2 La Filosovia,48.270005,46.007220,01-01-2022
3,1,1,MP 07: Naamsestraat 81,45.908501,44.373056,01-01-2022
4,1,2,MP 03: Naamsestraat 62 Taste,51.407297,50.094018,02-01-2022
...,...,...,...,...,...,...
2317,12,31,MP 04: His & Hears,55.134973,53.632457,31-12-2022
2318,12,31,MP 05: Calvariekapel KU Leuven,51.925297,50.133166,31-12-2022
2319,12,31,MP 06: Parkstraat 2 La Filosovia,50.342353,48.683298,31-12-2022
2320,12,31,MP 07: Naamsestraat 81,49.665543,47.970367,31-12-2022


In [571]:
# Apply the pipeline to the combined and collect monthly data
combined_monthly = pipeline_monthly.fit_transform(combined) # this df will also be used for modelling purpose later

combined_monthly.head(100000)

,month,description,lamax,laeq,date
0,1,MP 03: Naamsestraat 62 Taste,53.239009,51.727544,jan 2022
1,1,MP 05: Calvariekapel KU Leuven,50.374792,48.560197,jan 2022
2,1,MP 06: Parkstraat 2 La Filosovia,50.086348,48.274795,jan 2022
3,1,MP 07: Naamsestraat 81,48.800443,47.515371,jan 2022
4,2,MP 01: Naamsestraat 35 Maxim,57.287668,55.245502,feb 2022
...,...,...,...,...,...
77,12,MP 04: His & Hears,54.776651,53.133224,dec 2022
78,12,MP 05: Calvariekapel KU Leuven,52.407326,50.394638,dec 2022
79,12,MP 06: Parkstraat 2 La Filosovia,51.694696,49.915926,dec 2022
80,12,MP 07: Naamsestraat 81,50.699741,49.103125,dec 2022


In [572]:
# Define a function to standardize the data
def standardize_columns(df, columns_to_standardize):
    scaler = StandardScaler()
    standardized_values = scaler.fit_transform(df[columns_to_standardize])
    new_columns = [column + '_standardized' for column in columns_to_standardize]
    df[new_columns] = pd.DataFrame(standardized_values, columns=new_columns)
    return df

In [573]:
# Apply the function
combined_hourly = standardize_columns(combined_hourly, ['lamax', 'laeq'])
combined_daily = standardize_columns(combined_daily, ['lamax', 'laeq'])
combined_monthly = standardize_columns(combined_monthly, ['lamax', 'laeq'])

In [574]:
combined_hourly.head()

,month,day,hour,description,lamax,laeq,date,lamax_standardized,laeq_standardized
0,1,1,0,MP 03: Naamsestraat 62 Taste,60.322528,57.126833,00:00 01-01-2022,1.248969,1.044063
1,1,1,0,MP 05: Calvariekapel KU Leuven,53.230972,49.987639,00:00 01-01-2022,0.114661,-0.103638
2,1,1,0,MP 06: Parkstraat 2 La Filosovia,53.666056,50.752000,00:00 01-01-2022,0.184253,0.019241
3,1,1,0,MP 07: Naamsestraat 81,50.056861,47.440222,00:00 01-01-2022,-0.393044,-0.513162
4,1,1,1,MP 03: Naamsestraat 62 Taste,53.033583,50.853806,01:00 01-01-2022,0.083088,0.035608


In [575]:
combined_daily.head()

,month,day,description,lamax,laeq,date,lamax_standardized,laeq_standardized
0,1,1,MP 03: Naamsestraat 62 Taste,51.665242,49.992637,01-01-2022,-0.247251,-0.193655
1,1,1,MP 05: Calvariekapel KU Leuven,48.747476,46.504067,01-01-2022,-1.093428,-1.245260
2,1,1,MP 06: Parkstraat 2 La Filosovia,48.270005,46.007220,01-01-2022,-1.231899,-1.395031
3,1,1,MP 07: Naamsestraat 81,45.908501,44.373056,01-01-2022,-1.916755,-1.887638
4,1,2,MP 03: Naamsestraat 62 Taste,51.407297,50.094018,02-01-2022,-0.322058,-0.163094


In [576]:
combined_monthly.head()

,month,description,lamax,laeq,date,lamax_standardized,laeq_standardized
0,1,MP 03: Naamsestraat 62 Taste,53.239009,51.727544,jan 2022,0.238516,0.391739
1,1,MP 05: Calvariekapel KU Leuven,50.374792,48.560197,jan 2022,-0.780260,-0.796668
2,1,MP 06: Parkstraat 2 La Filosovia,50.086348,48.274795,jan 2022,-0.882857,-0.903753
3,1,MP 07: Naamsestraat 81,48.800443,47.515371,jan 2022,-1.340242,-1.188693
4,2,MP 01: Naamsestraat 35 Maxim,57.287668,55.245502,feb 2022,1.678588,1.711697


In [577]:
# check whether there are missing values left
print(combined_hourly.isnull().sum())
print(combined_daily.isnull().sum())
print(combined_monthly.isnull().sum())

month                 0
day                   0
hour                  0
description           0
lamax                 0
laeq                  0
date                  0
lamax_standardized    0
laeq_standardized     0
dtype: int64
month                 0
day                   0
description           0
lamax                 0
laeq                  0
date                  0
lamax_standardized    0
laeq_standardized     0
dtype: int64
month                 0
description           0
lamax                 0
laeq                  0
date                  0
lamax_standardized    0
laeq_standardized     0
dtype: int64


In [162]:
'''
# exporting file (only needs to be run one time so comment it out)
combined_hourly.to_csv('hourly_noisedata_2022.csv', index=False)  
combined_daily.to_csv('daily_noisedata_2022.csv', index=False) 
combined_monthly.to_csv('monthly_noisedata_2022.csv', index=False) 
'''

In [578]:
# drop unnecessary column for later use and reduce memory usage
combined_hourly = combined_hourly.drop(['date','lamax_standardized','laeq_standardized'], axis=1)
combined_daily = combined_daily.drop(['date','lamax_standardized','laeq_standardized'], axis=1)
combined_monthly = combined_monthly.drop(['date','lamax_standardized','laeq_standardized'], axis=1)

### Preprocessing for modelling purpose

In [579]:
# Load weather data
weather_data = pd.read_csv("../Data/hourly_weatherdata_2022.csv", header = 0, sep=',')
weather_data.head()

,Month,Day,Hour,LC_RAININ,LC_DAILYRAIN,LC_WINDDIR,LC_WINDSPEED,LC_TEMP_QCL3
0,1,1,0,0.000023,0.002997,-33.566358,1.487099,15.513391
1,1,1,1,0.000019,0.002174,-29.188272,1.465571,15.770757
2,1,1,2,0.000003,0.000360,-18.197324,0.389565,13.100358
3,1,1,3,0.000007,0.000000,-16.227891,0.222602,12.669197
4,1,1,4,0.000009,0.000000,-13.710884,0.217194,12.520271


In [580]:
len(weather_data) # the length of a complete dataset at 1 location should be 8760

8760

##### For MP 03: Taste

- Step 1: Get the hourly noise data at MP03 and find the missing values

In [581]:
# Select the noise data from MP 03
noise_MP03 = combined_hourly[combined_hourly["description"] == "MP 03: Naamsestraat 62 Taste"]
len(noise_MP03) # full length should be 8760, so there is 1 missing value in the data

8759

In [582]:
# To spot where the missing values are
noise_month = noise_MP03[noise_MP03["month"] == 3]  
noise_day = noise_month[noise_month["day"] == 27]
len(noise_day) # the result is 23 instead of 24h, so the missing value is on Mar 27th

23

In [583]:
print(noise_day) # 2AM is the missing value

      month  day  hour                   description      lamax       laeq
9420      3   27     0  MP 03: Naamsestraat 62 Taste  52.364694  50.373139
9426      3   27     1  MP 03: Naamsestraat 62 Taste  52.617361  50.700694
9432      3   27     3  MP 03: Naamsestraat 62 Taste  51.685361  49.772833
9438      3   27     4  MP 03: Naamsestraat 62 Taste  50.847083  49.127972
9444      3   27     5  MP 03: Naamsestraat 62 Taste  48.657611  46.836694
9450      3   27     6  MP 03: Naamsestraat 62 Taste  52.940667  51.199833
9456      3   27     7  MP 03: Naamsestraat 62 Taste  50.119833  48.395333
9462      3   27     8  MP 03: Naamsestraat 62 Taste  48.312194  46.660639
9468      3   27     9  MP 03: Naamsestraat 62 Taste  48.338556  46.461667
9474      3   27    10  MP 03: Naamsestraat 62 Taste  67.138750  64.770000
9480      3   27    11  MP 03: Naamsestraat 62 Taste  60.786417  58.523778
9486      3   27    12  MP 03: Naamsestraat 62 Taste  64.480639  62.336500
9492      3   27    13  M

- Step 2: Impute the missing value

In [584]:
# Add a new row to a dataframe
mar_27_2 = {'month':3, 'day':27, 'hour':2, 'description':'MP 03: Naamsestraat 62 Taste', 'lamax':np.nan, 'laeq':np.nan}
noise_MP03 = pd.concat([noise_MP03, pd.DataFrame(mar_27_2, index=[0])], ignore_index=True)
noise_MP03 = noise_MP03.sort_values(['month','day','hour']).reset_index().drop('index', axis = 1)
len(noise_MP03)

8760

In [585]:
# Forward impute single observation in noise series on March 27 2AM
noise_MP03 = noise_MP03.ffill()

In [586]:
# Check whether there are missing values left
print(noise_MP03.isnull().sum())

month          0
day            0
hour           0
description    0
lamax          0
laeq           0
dtype: int64


- Step 3: Add extra columns we need

In [587]:
# Variable 1: weekday
# day of week one hot encoding. Go for 24 hours, skip 144. Year starts on a saturday
lst = [6, 7, 1, 2, 3, 4, 5]
lst_new = list(np.repeat(lst,24))
lst_newnew = lst_new * 53
day = np.array(lst_newnew[0:8760])

weekday = pd.get_dummies(day, prefix=['sat', 'sun', 'mon', 'tue', 'wed', 'thu', 'fri'])

In [588]:
# Variable 2: normal holiday
holiday_df = pd.read_csv("../Data/Holidays.csv", header = 0, sep=',')
holiday_list = holiday_df['Date'].tolist()

locale.setlocale(locale.LC_TIME, 'nl_NL')

formatted_holidays = []
for date_str in holiday_list:
    date_obj = datetime.strptime(date_str, "%d %b")
    formatted_holidays.append(date_obj)

formatted_holidays = [date_obj.strftime("%m-%d") for date_obj in formatted_holidays]
prefix = '2022-'
formatted_holidays = [prefix + i for i in formatted_holidays]


start_date = '2022-01-01 00:00:00'
end_date = '2022-12-31 23:59:59'
index = pd.date_range(start=start_date, end=end_date, freq='H')
df = pd.DataFrame(index=index)

df['Holiday'] = 0

for holiday in formatted_holidays:
    df.loc[holiday, 'Holiday'] = 1

holiday = pd.Series(df['Holiday'])
holiday_series = holiday.reset_index(drop = True)
len(holiday_series)

8760

In [589]:
# Variable 3: KUL hoilday
# kuleuven calendar
#https://www.kuleuven.be/over-kuleuven/kalenders/kalenders-21-22/academische-kalender-2021-2022-ku-leuven-campus-leuven
#https://www.kuleuven.be/over-kuleuven/kalenders/kalenders-22-23/ku-leuven-leuven
holidays_unif_begin = [[1,1],[4,2],[5,28],[12,24]] #month then day
holidays_unif_end = [[2,13],[4,18],[9,26],[12,31]]

# create a copy of noise data
noise_MP03_withkulholiday = noise_MP03.copy()
# create a new column for holiday indicator
noise_MP03_withkulholiday['is_kul_holiday'] = False

# iterate over each row in the DataFrame
for index, row in noise_MP03_withkulholiday.iterrows():
    result_month = row['month']
    result_day = row['day']
    
    # check if the current date is a holiday
    for begin, end in zip(holidays_unif_begin, holidays_unif_end):
        if begin[0] <= result_month <= end[0] and begin[1] <= result_day <= end[1]:
            noise_MP03_withkulholiday.at[index, 'is_kul_holiday'] = True
            break

In [590]:
noise_MP03_withkulholiday.head(2000)

,month,day,hour,description,lamax,laeq,is_kul_holiday
0,1,1,0,MP 03: Naamsestraat 62 Taste,60.322528,57.126833,True
1,1,1,1,MP 03: Naamsestraat 62 Taste,53.033583,50.853806,True
2,1,1,2,MP 03: Naamsestraat 62 Taste,52.173702,50.049903,True
3,1,1,3,MP 03: Naamsestraat 62 Taste,50.821311,48.964907,True
4,1,1,4,MP 03: Naamsestraat 62 Taste,49.134851,47.285893,True
...,...,...,...,...,...,...,...
1995,3,25,3,MP 03: Naamsestraat 62 Taste,54.462889,51.988222,False
1996,3,25,4,MP 03: Naamsestraat 62 Taste,54.513389,51.867944,False
1997,3,25,5,MP 03: Naamsestraat 62 Taste,53.198778,50.924750,False
1998,3,25,6,MP 03: Naamsestraat 62 Taste,52.823917,51.268417,False


- Step 4: Define feature matrix and target vector

In [591]:
# Define feature matrix X
# convert columns in dfs to series
temp_series = pd.Series(weather_data['LC_TEMP_QCL3'])
rain_series = pd.Series(weather_data['LC_RAININ'])
rain_cumul_series = pd.Series(weather_data['LC_DAILYRAIN'])
windspeed_series = pd.Series(weather_data['LC_WINDSPEED'])
month = pd.Series(weather_data['Month'])
day = pd.Series(weather_data['Day'])
hour = pd.Series(weather_data['Hour'])
kul_hoilday = pd.Series(noise_MP03_withkulholiday['is_kul_holiday'])

In [592]:
# combine into a feature matrix
features = pd.concat([hour, weekday, holiday_series, kul_hoilday, temp_series, rain_series, rain_cumul_series, windspeed_series], axis=1)
display(features)

,Hour,"['sat', 'sun', 'mon', 'tue', 'wed', 'thu', 'fri']_1","['sat', 'sun', 'mon', 'tue', 'wed', 'thu', 'fri']_2","['sat', 'sun', 'mon', 'tue', 'wed', 'thu', 'fri']_3","['sat', 'sun', 'mon', 'tue', 'wed', 'thu', 'fri']_4","['sat', 'sun', 'mon', 'tue', 'wed', 'thu', 'fri']_5","['sat', 'sun', 'mon', 'tue', 'wed', 'thu', 'fri']_6","['sat', 'sun', 'mon', 'tue', 'wed', 'thu', 'fri']_7",Holiday,is_kul_holiday,LC_TEMP_QCL3,LC_RAININ,LC_DAILYRAIN,LC_WINDSPEED
0,0,False,False,False,False,False,True,False,1,True,15.513391,0.000023,0.002997,1.487099
1,1,False,False,False,False,False,True,False,1,True,15.770757,0.000019,0.002174,1.465571
2,2,False,False,False,False,False,True,False,1,True,13.100358,0.000003,0.000360,0.389565
3,3,False,False,False,False,False,True,False,1,True,12.669197,0.000007,0.000000,0.222602
4,4,False,False,False,False,False,True,False,1,True,12.520271,0.000009,0.000000,0.217194
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,19,False,False,False,False,False,True,False,1,True,14.964675,0.000074,0.002981,1.200756
8756,20,False,False,False,False,False,True,False,1,True,15.006786,0.000077,0.002981,1.364861
8757,21,False,False,False,False,False,True,False,1,True,15.320673,0.000077,0.002988,1.473519
8758,22,False,False,False,False,False,True,False,1,True,15.483860,0.000074,0.002991,1.416481


In [593]:
# Define target vector y
noise_series_MP03 = pd.Series(noise_MP03_withkulholiday['laeq']) # choose laeq as target vector

- Step 5: Split the training and test dataset

In [594]:
# Split training and test data randomly
X_train_MP03, X_test_MP03, y_train_MP03, y_test_MP03 = train_test_split(features, noise_series_MP03,
    test_size=0.2)

In [595]:
X_train_MP03.head()

,Hour,"['sat', 'sun', 'mon', 'tue', 'wed', 'thu', 'fri']_1","['sat', 'sun', 'mon', 'tue', 'wed', 'thu', 'fri']_2","['sat', 'sun', 'mon', 'tue', 'wed', 'thu', 'fri']_3","['sat', 'sun', 'mon', 'tue', 'wed', 'thu', 'fri']_4","['sat', 'sun', 'mon', 'tue', 'wed', 'thu', 'fri']_5","['sat', 'sun', 'mon', 'tue', 'wed', 'thu', 'fri']_6","['sat', 'sun', 'mon', 'tue', 'wed', 'thu', 'fri']_7",Holiday,is_kul_holiday,LC_TEMP_QCL3,LC_RAININ,LC_DAILYRAIN,LC_WINDSPEED
6103,7,True,False,False,False,False,False,False,0,False,11.578077,0.000000,0.000000,0.001852
1172,20,False,False,False,False,True,False,False,0,False,8.914201,0.000000,0.002296,2.575442
402,18,True,False,False,False,False,False,False,0,False,7.428357,0.000009,0.000194,0.117007
4296,0,False,False,True,False,False,False,False,0,False,16.520262,0.000000,0.000019,0.018117
2237,5,True,False,False,False,False,False,False,0,True,2.505604,0.000009,0.000000,0.278272


In [333]:
'''
# exporting file (only needs to be run one time so comment it out)
X_train_MP03.to_csv('X_train_MP03.csv', index=False)  
X_test_MP03.to_csv('X_test_MP03.csv', index=False)
y_train_MP03.to_csv('y_train_MP03.csv', index=False)
y_test_MP03.to_csv('y_test_MP03.csv', index=False)
'''

- Step 6: Outlier dection

In [596]:
# Check outlier (using MAD only on X training data)
def detect_outliers(df, threshold=3.5):
    outliers = pd.DataFrame()

    for column in df.columns:
        median = df[column].median()
        mad = np.median(np.abs(df[column] - median))
        modified_z_scores = 0.6745 * (df[column] - median) / mad
        column_outliers = df[np.abs(modified_z_scores) > threshold]

        # append the column outliers to the overall outliers dataframe
        outliers = pd.concat([outliers, column_outliers])

    return outliers

# Apply the function
outliers_MP03 = detect_outliers(X_train_MP03)
outliers_MP03.head()

,Hour,"['sat', 'sun', 'mon', 'tue', 'wed', 'thu', 'fri']_1","['sat', 'sun', 'mon', 'tue', 'wed', 'thu', 'fri']_2","['sat', 'sun', 'mon', 'tue', 'wed', 'thu', 'fri']_3","['sat', 'sun', 'mon', 'tue', 'wed', 'thu', 'fri']_4","['sat', 'sun', 'mon', 'tue', 'wed', 'thu', 'fri']_5","['sat', 'sun', 'mon', 'tue', 'wed', 'thu', 'fri']_6","['sat', 'sun', 'mon', 'tue', 'wed', 'thu', 'fri']_7",Holiday,is_kul_holiday,LC_TEMP_QCL3,LC_RAININ,LC_DAILYRAIN,LC_WINDSPEED
6103,7,True,False,False,False,False,False,False,0,False,11.578077,0.000000,0.000000,0.001852
402,18,True,False,False,False,False,False,False,0,False,7.428357,0.000009,0.000194,0.117007
2237,5,True,False,False,False,False,False,False,0,True,2.505604,0.000009,0.000000,0.278272
1562,2,True,False,False,False,False,False,False,0,False,-0.521954,0.000000,0.000031,0.079099
2083,19,True,False,False,False,False,False,False,0,False,15.574256,0.000000,0.000031,0.087551


##### For MP05: Calvariekapel

- Step 1: Get the hourly noise data at MP05 and find the missing values

In [597]:
# Select the noise data from MP 05
noise_MP05 = combined_hourly[combined_hourly["description"] == "MP 05: Calvariekapel KU Leuven"]
len(noise_MP05) # full length should be 8760, so there are 2 missing values in the data

8758

In [598]:
# To spot where the missing values are
# missing value 1
noise_month_3 = noise_MP05[noise_MP05["month"] == 3]  
noise_day_27 = noise_month_3[noise_month_3["day"] == 27]
len(noise_day_27) # the result is 23 instead of 24h, so the missing value is on Mar 27th

23

In [599]:
print(noise_day_27) # 2AM is the missing value

      month  day  hour                     description      lamax       laeq
9421      3   27     0  MP 05: Calvariekapel KU Leuven  49.612222  46.969528
9427      3   27     1  MP 05: Calvariekapel KU Leuven  46.180222  43.672694
9433      3   27     3  MP 05: Calvariekapel KU Leuven  45.319944  42.577722
9439      3   27     4  MP 05: Calvariekapel KU Leuven  51.102222  49.020944
9445      3   27     5  MP 05: Calvariekapel KU Leuven  42.675222  40.485583
9451      3   27     6  MP 05: Calvariekapel KU Leuven  49.373061  47.456436
9457      3   27     7  MP 05: Calvariekapel KU Leuven  46.376500  43.755417
9463      3   27     8  MP 05: Calvariekapel KU Leuven  42.629639  40.194444
9469      3   27     9  MP 05: Calvariekapel KU Leuven  44.326722  41.335167
9475      3   27    10  MP 05: Calvariekapel KU Leuven  56.256500  53.573611
9481      3   27    11  MP 05: Calvariekapel KU Leuven  54.437250  51.706583
9487      3   27    12  MP 05: Calvariekapel KU Leuven  57.361972  54.724778

In [600]:
# missing value 2
noise_month_10 = noise_MP05[noise_MP05["month"] == 10]  
noise_day_2 = noise_month_10[noise_month_10["day"] == 2]
len(noise_day_2) # the result is 23 instead of 24h, so the missing value is on Mar 27th

23

In [601]:
print(noise_day_2) # 9AM is the missing value

       month  day  hour                     description      lamax       laeq
40316     10    2     0  MP 05: Calvariekapel KU Leuven  50.911028  48.448139
40322     10    2     1  MP 05: Calvariekapel KU Leuven  48.630139  46.312111
40328     10    2     2  MP 05: Calvariekapel KU Leuven  47.109028  44.837139
40334     10    2     3  MP 05: Calvariekapel KU Leuven  47.087389  44.868361
40340     10    2     4  MP 05: Calvariekapel KU Leuven  47.495944  45.138917
40346     10    2     5  MP 05: Calvariekapel KU Leuven  46.527944  44.103250
40352     10    2     6  MP 05: Calvariekapel KU Leuven  45.353250  43.197861
40358     10    2     7  MP 05: Calvariekapel KU Leuven  47.590500  45.392861
40364     10    2     8  MP 05: Calvariekapel KU Leuven  49.039574  47.293700
40375     10    2    10  MP 05: Calvariekapel KU Leuven  53.212595  51.430170
40381     10    2    11  MP 05: Calvariekapel KU Leuven  54.616222  52.670306
40387     10    2    12  MP 05: Calvariekapel KU Leuven  53.0801

- Step 2: Impute the missing values

In [602]:
# Add a new row to a dataframe
mar_27_2 = {'month':3, 'day':27, 'hour':2, 'description':'MP 05: Calvariekapel KU Leuven', 'lamax':np.nan, 'laeq':np.nan}
oct_2_9 = {'month':10, 'day':2, 'hour':9, 'description':'MP 05: Calvariekapel KU Leuven', 'lamax':np.nan, 'laeq':np.nan}
noise_MP05 = pd.concat([noise_MP05, pd.DataFrame(mar_27_2, index=[0]), pd.DataFrame(oct_2_9, index=[0])], ignore_index=True)
noise_MP05 = noise_MP05.sort_values(['month','day','hour']).reset_index().drop('index', axis = 1)
len(noise_MP05)

8760

In [603]:
#Forward impute single observation in noise series on March 27 2AM and Oct 2 9AM
noise_MP05 = noise_MP05.ffill()

In [604]:
# check whether there are missing values left
print(noise_MP03.isnull().sum())

month          0
day            0
hour           0
description    0
lamax          0
laeq           0
dtype: int64


- Step 3: Add extra columns we need

In [605]:
# Add a new kul holiday dummy variable
# create a copy of noise data
noise_MP05_withkulholiday = noise_MP05.copy()
# create a new column for holiday indicator
noise_MP05_withkulholiday['is_kul_holiday'] = False

# iterate over each row in the DataFrame
for index, row in noise_MP05_withkulholiday.iterrows():
    result_month = row['month']
    result_day = row['day']
    
    # check if the current date is a holiday
    for begin, end in zip(holidays_unif_begin, holidays_unif_end):
        if begin[0] <= result_month <= end[0] and begin[1] <= result_day <= end[1]:
            noise_MP05_withkulholiday.at[index, 'is_kul_holiday'] = True
            break

In [606]:
noise_MP05_withkulholiday.head(2000)

,month,day,hour,description,lamax,laeq,is_kul_holiday
0,1,1,0,MP 05: Calvariekapel KU Leuven,53.230972,49.987639,True
1,1,1,1,MP 05: Calvariekapel KU Leuven,53.599639,50.578806,True
2,1,1,2,MP 05: Calvariekapel KU Leuven,51.078083,47.974361,True
3,1,1,3,MP 05: Calvariekapel KU Leuven,48.187778,45.575083,True
4,1,1,4,MP 05: Calvariekapel KU Leuven,44.372528,41.675250,True
...,...,...,...,...,...,...,...
1995,3,25,3,MP 05: Calvariekapel KU Leuven,52.805500,49.479861,False
1996,3,25,4,MP 05: Calvariekapel KU Leuven,51.988972,48.792389,False
1997,3,25,5,MP 05: Calvariekapel KU Leuven,48.871139,46.405167,False
1998,3,25,6,MP 05: Calvariekapel KU Leuven,50.657861,48.918556,False


- Step 4: Selet noise data from jun 1st 0h to jun 14th 13h

In [607]:
# Get start index for both noise and old feature
index_start_n_f = weather_data.loc[(weather_data['Month'] == 6) & (weather_data['Day'] == 1) & (weather_data['Hour'] == 0)].index
print(index_start_n_f)

Index([3624], dtype='int64')


In [608]:
# Get end index for both noise and old feature
index_end_n_f = weather_data.loc[(weather_data['Month'] == 6) & (weather_data['Day'] == 14) & (weather_data['Hour'] == 13)].index
print(index_end_n_f)

Index([3949], dtype='int64')


In [609]:
index_start_n_f = 3624
index_end_n_f = 3949
noise_jun1to14 = noise_MP05_withkulholiday.iloc[index_start_n_f:index_end_n_f+1]
len(noise_jun1to14)

326

In [610]:
noise_jun1to14.head(500)

,month,day,hour,description,lamax,laeq,is_kul_holiday
3624,6,1,0,MP 05: Calvariekapel KU Leuven,46.389500,43.854917,False
3625,6,1,1,MP 05: Calvariekapel KU Leuven,42.325056,39.724889,False
3626,6,1,2,MP 05: Calvariekapel KU Leuven,40.794889,38.388694,False
3627,6,1,3,MP 05: Calvariekapel KU Leuven,40.153528,38.418528,False
3628,6,1,4,MP 05: Calvariekapel KU Leuven,41.507194,39.652750,False
...,...,...,...,...,...,...,...
3945,6,14,9,MP 05: Calvariekapel KU Leuven,57.855417,56.265333,False
3946,6,14,10,MP 05: Calvariekapel KU Leuven,59.364250,57.658889,False
3947,6,14,11,MP 05: Calvariekapel KU Leuven,58.431417,56.621444,False
3948,6,14,12,MP 05: Calvariekapel KU Leuven,57.068556,55.056944,False


- Step 5: Select corresponding feature from jun 1st 0h to jun 14th 13h

In [611]:
features_jun1to14 = features.iloc[index_start_n_f:index_end_n_f+1] # the index can be reused as it's full length data
features_jun1to14 = features_jun1to14.reset_index()
len(features_jun1to14)

326

In [612]:
features_jun1to14.head(500)

,index,Hour,"['sat', 'sun', 'mon', 'tue', 'wed', 'thu', 'fri']_1","['sat', 'sun', 'mon', 'tue', 'wed', 'thu', 'fri']_2","['sat', 'sun', 'mon', 'tue', 'wed', 'thu', 'fri']_3","['sat', 'sun', 'mon', 'tue', 'wed', 'thu', 'fri']_4","['sat', 'sun', 'mon', 'tue', 'wed', 'thu', 'fri']_5","['sat', 'sun', 'mon', 'tue', 'wed', 'thu', 'fri']_6","['sat', 'sun', 'mon', 'tue', 'wed', 'thu', 'fri']_7",Holiday,is_kul_holiday,LC_TEMP_QCL3,LC_RAININ,LC_DAILYRAIN,LC_WINDSPEED
0,3624,0,False,False,True,False,False,False,False,0,False,10.378574,0.0,0.0,0.007145
1,3625,1,False,False,True,False,False,False,False,0,False,9.603619,0.0,0.0,0.020062
2,3626,2,False,False,True,False,False,False,False,0,False,9.042755,0.0,0.0,0.014985
3,3627,3,False,False,True,False,False,False,False,0,False,8.435343,0.0,0.0,0.013488
4,3628,4,False,False,True,False,False,False,False,0,False,7.976961,0.0,0.0,0.025355
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
321,3945,9,False,True,False,False,False,False,False,0,False,15.426370,0.0,0.0,0.167515
322,3946,10,False,True,False,False,False,False,False,0,False,16.823099,0.0,0.0,0.224861
323,3947,11,False,True,False,False,False,False,False,0,False,17.979838,0.0,0.0,0.256651
324,3948,12,False,True,False,False,False,False,False,0,False,19.256854,0.0,0.0,0.283843


- Step 6: Preprocess traffic data

In [680]:
# Load the traffic data
traffic_preprocessed = pd.read_csv('../Data/hourly_traffic_2022.csv', header = 0, sep=',')
traffic_preprocessed.head(410)

,date,heavy,car,bike,pedestrian,timezone,description,month,day,hour,year
0,00:00 01-06-2022,0.000000,0.000000,0.000000,0.000000,Europe/Brussels,MP 05: Calvariekapel KU Leuven,6,1,0,2022
1,01:00 01-06-2022,0.000000,0.000000,0.000000,0.000000,Europe/Brussels,MP 05: Calvariekapel KU Leuven,6,1,1,2022
2,02:00 01-06-2022,0.000000,0.000000,0.000000,0.000000,Europe/Brussels,MP 05: Calvariekapel KU Leuven,6,1,2,2022
3,03:00 01-06-2022,20.532319,6.844106,20.532319,0.000000,Europe/Brussels,MP 05: Calvariekapel KU Leuven,6,1,3,2022
4,04:00 01-06-2022,37.894737,87.157895,56.842105,2.526316,Europe/Brussels,MP 05: Calvariekapel KU Leuven,6,1,4,2022
...,...,...,...,...,...,...,...,...,...,...,...
405,10:00 31-08-2022,1.410658,236.990596,366.771160,8.463950,Europe/Brussels,MP 05: Calvariekapel KU Leuven,8,31,10,2022
406,11:00 31-08-2022,8.530806,210.426540,227.488152,0.000000,Europe/Brussels,MP 05: Calvariekapel KU Leuven,8,31,11,2022
407,12:00 31-08-2022,8.834356,229.693252,260.613497,4.417178,Europe/Brussels,MP 05: Calvariekapel KU Leuven,8,31,12,2022
408,13:00 31-08-2022,19.823789,272.246696,288.105727,68.722467,Europe/Brussels,MP 05: Calvariekapel KU Leuven,8,31,13,2022


- Step 7: Selet traffic data from jun 1st 0h to jun 14th 13h

In [616]:
# Get start index for traffic data
index_start_t = traffic_preprocessed.loc[(traffic_preprocessed['month'] == 6) & (traffic_preprocessed['day'] == 1) & (traffic_preprocessed['hour'] == 0)].index
print(index_start_t)

Index([0], dtype='int64')


In [617]:
# Get end index for traffic data
index_end_t = traffic_preprocessed.loc[(traffic_preprocessed['month'] == 6) & (traffic_preprocessed['day'] == 14) & (traffic_preprocessed['hour'] == 13)].index
print(index_end_t)

Index([325], dtype='int64')


In [618]:
index_start_t = 0
index_end_t = 325
traffic_jun1to14 = traffic_preprocessed.iloc[index_start_t:index_end_t+1]
len(traffic_jun1to14)

326

In [619]:
traffic_jun1to14.head(500)

,date,heavy,car,bike,pedestrian,timezone,description,month,day,hour,year
0,00:00 01-06-2022,0.000000,0.000000,0.000000,0.000000,Europe/Brussels,MP 05: Calvariekapel KU Leuven,6,1,0,2022
1,01:00 01-06-2022,0.000000,0.000000,0.000000,0.000000,Europe/Brussels,MP 05: Calvariekapel KU Leuven,6,1,1,2022
2,02:00 01-06-2022,0.000000,0.000000,0.000000,0.000000,Europe/Brussels,MP 05: Calvariekapel KU Leuven,6,1,2,2022
3,03:00 01-06-2022,20.532319,6.844106,20.532319,0.000000,Europe/Brussels,MP 05: Calvariekapel KU Leuven,6,1,3,2022
4,04:00 01-06-2022,37.894737,87.157895,56.842105,2.526316,Europe/Brussels,MP 05: Calvariekapel KU Leuven,6,1,4,2022
...,...,...,...,...,...,...,...,...,...,...,...
321,09:00 14-06-2022,13.798390,274.587965,389.114603,16.558068,Europe/Brussels,MP 05: Calvariekapel KU Leuven,6,14,9,2022
322,10:00 14-06-2022,3.816254,179.363958,251.872792,10.176678,Europe/Brussels,MP 05: Calvariekapel KU Leuven,6,14,10,2022
323,11:00 14-06-2022,45.346396,234.289713,264.520644,3.778866,Europe/Brussels,MP 05: Calvariekapel KU Leuven,6,14,11,2022
324,12:00 14-06-2022,43.353404,173.413617,248.448356,0.000000,Europe/Brussels,MP 05: Calvariekapel KU Leuven,6,14,12,2022


In [620]:
# drop unwanted columns
columns_to_keep = ['heavy', 'car', 'bike', 'pedestrian']
columns_to_drop = set(traffic_jun1to14.columns) - set(columns_to_keep)
traffic_jun1to14 = traffic_jun1to14.drop(columns=columns_to_drop)

In [621]:
traffic_jun1to14.head()

,heavy,car,bike,pedestrian
0,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000
3,20.532319,6.844106,20.532319,0.000000
4,37.894737,87.157895,56.842105,2.526316


- Step 8: Define feature matrix and target vector

In [622]:
# Define feature matrix X
# Create a new feature matrix for MP05
features_MP05 = pd.concat([features_jun1to14, traffic_jun1to14], axis=1, join='outer')
features_MP05.head()

,index,Hour,"['sat', 'sun', 'mon', 'tue', 'wed', 'thu', 'fri']_1","['sat', 'sun', 'mon', 'tue', 'wed', 'thu', 'fri']_2","['sat', 'sun', 'mon', 'tue', 'wed', 'thu', 'fri']_3","['sat', 'sun', 'mon', 'tue', 'wed', 'thu', 'fri']_4","['sat', 'sun', 'mon', 'tue', 'wed', 'thu', 'fri']_5","['sat', 'sun', 'mon', 'tue', 'wed', 'thu', 'fri']_6","['sat', 'sun', 'mon', 'tue', 'wed', 'thu', 'fri']_7",Holiday,is_kul_holiday,LC_TEMP_QCL3,LC_RAININ,LC_DAILYRAIN,LC_WINDSPEED,heavy,car,bike,pedestrian
0,3624,0,False,False,True,False,False,False,False,0,False,10.378574,0.0,0.0,0.007145,0.000000,0.000000,0.000000,0.000000
1,3625,1,False,False,True,False,False,False,False,0,False,9.603619,0.0,0.0,0.020062,0.000000,0.000000,0.000000,0.000000
2,3626,2,False,False,True,False,False,False,False,0,False,9.042755,0.0,0.0,0.014985,0.000000,0.000000,0.000000,0.000000
3,3627,3,False,False,True,False,False,False,False,0,False,8.435343,0.0,0.0,0.013488,20.532319,6.844106,20.532319,0.000000
4,3628,4,False,False,True,False,False,False,False,0,False,7.976961,0.0,0.0,0.025355,37.894737,87.157895,56.842105,2.526316


In [623]:
# Define target vector y
noise_series_MP05 = pd.Series(noise_jun1to14['laeq']) # choose laeq as target vector

In [624]:
len(noise_series_MP05)

326

- Step 9: Split the training and test dataset

In [625]:
# Split training and test data randomly
X_train_MP05, X_test_MP05, y_train_MP05, y_test_MP05 = train_test_split(features_MP05, noise_series_MP05,
    test_size=0.2)

In [487]:
'''
# exporting file (only needs to be run one time so comment it out)
X_train_MP05.to_csv('X_train_MP05.csv', index=False)  
X_test_MP05.to_csv('X_test_MP05.csv', index=False)
y_train_MP05.to_csv('y_train_MP05.csv', index=False)
y_test_MP05.to_csv('y_test_MP05.csv', index=False)
'''

- Step 10: Outlier detection

In [626]:
# Check outlier (using MAD only on X training data)
def detect_outliers(df, threshold=3.5):
    outliers = pd.DataFrame()

    for column in df.columns:
        median = df[column].median()
        mad = np.median(np.abs(df[column] - median))
        modified_z_scores = 0.6745 * (df[column] - median) / mad
        column_outliers = df[np.abs(modified_z_scores) > threshold]

        # append the column outliers to the overall outliers dataframe
        outliers = pd.concat([outliers, column_outliers])

    return outliers

# Apply the function
outliers_MP05 = detect_outliers(X_train_MP05)
outliers_MP05.head()

,index,Hour,"['sat', 'sun', 'mon', 'tue', 'wed', 'thu', 'fri']_1","['sat', 'sun', 'mon', 'tue', 'wed', 'thu', 'fri']_2","['sat', 'sun', 'mon', 'tue', 'wed', 'thu', 'fri']_3","['sat', 'sun', 'mon', 'tue', 'wed', 'thu', 'fri']_4","['sat', 'sun', 'mon', 'tue', 'wed', 'thu', 'fri']_5","['sat', 'sun', 'mon', 'tue', 'wed', 'thu', 'fri']_6","['sat', 'sun', 'mon', 'tue', 'wed', 'thu', 'fri']_7",Holiday,is_kul_holiday,LC_TEMP_QCL3,LC_RAININ,LC_DAILYRAIN,LC_WINDSPEED,heavy,car,bike,pedestrian
299,3923,11,True,False,False,False,False,False,False,0,False,17.612261,0.000000,0.000000,0.479907,26.808511,209.361702,259.148936,8.936170
300,3924,12,True,False,False,False,False,False,False,0,False,18.990428,0.000000,0.000000,0.573426,24.203822,185.987261,189.808917,5.095541
142,3766,22,True,False,False,False,False,False,False,1,False,14.356561,0.000005,0.000139,0.010880,0.000000,0.000000,0.000000,0.000000
301,3925,13,True,False,False,False,False,False,False,0,False,19.379951,0.000000,0.000000,0.541420,17.293835,227.702162,164.291433,5.764612
122,3746,2,True,False,False,False,False,False,False,1,False,14.992525,0.000008,0.000037,0.353318,0.000000,0.000000,0.000000,0.000000


## Noise events

#### Load events data

In [627]:
# Load the events data
event_mp01 = pd.read_csv('../Data/csv_results_41_255439_mp-01-naamsestraat-35-maxim.csv',delimiter=';')
event_mp02 = pd.read_csv('../Data/csv_results_41_255440_mp-02-naamsestraat-57-xior.csv',delimiter=';')
event_mp03 = pd.read_csv('../Data/csv_results_41_255441_mp-03-naamsestraat-62-taste.csv',delimiter=';')
event_mp04 = pd.read_csv('../Data/csv_results_41_303910_mp-04-his-hears.csv',delimiter=';')
event_mp05 = pd.read_csv('../Data/csv_results_41_255442_mp-05-calvariekapel-ku-leuven.csv',delimiter=';')
event_mp06 = pd.read_csv('../Data/csv_results_41_255443_mp-06-parkstraat-2-la-filosovia.csv',delimiter=';')
event_mp07 = pd.read_csv('../Data/csv_results_41_255444_mp-07-naamsestraat-81.csv',delimiter=';')
event_mp08Vrijthof = pd.read_csv('../Data/csv_results_41_280324_mp08bis---vrijthof.csv',delimiter=';')

#### Preprocess events data for merging purpose

- Build pipeline

In [628]:
# Step 1: Concatenate data
def concatenate_datasets(dfs):
    return pd.concat(dfs, ignore_index=True)

# Step 2: Drop columns
def drop_columns(df):
    columns_to_keep = ['description', 'result_timestamp', 'noise_event_laeq_primary_detected_certainty', 'noise_event_laeq_primary_detected_class']
    columns_to_drop = set(df.columns) - set(columns_to_keep)
    return df.drop(columns=columns_to_drop)

# Step 3: Add a column of certainty in percentage form (in string form) 
def percentage_column(df):
    df['certainty_percentage'] = df['noise_event_laeq_primary_detected_certainty'].apply(lambda x: f"{x}%" if not pd.isnull(x) else np.nan)
    return df

# Step 4: extract time from 'result_timestamp' 
def extract_time(df):
    df['result_timestamp'] = pd.to_datetime(df['result_timestamp'], format='%d/%m/%Y %H:%M:%S.%f')
    df['month'] = df['result_timestamp'].dt.month
    df['day'] = df['result_timestamp'].dt.day
    df['hour'] = df['result_timestamp'].dt.hour
    df['minute'] = df['result_timestamp'].dt.minute
    df['second'] = df['result_timestamp'].dt.second
    df['milliseconds'] = df['result_timestamp'].dt.microsecond // 1000
    return df

# Define the pipeline
pipeline_merge_event = Pipeline([
    ('concatenate_datasets', FunctionTransformer(concatenate_datasets)),
    ('drop_columns', FunctionTransformer(drop_columns)),
    ('percentage_column', FunctionTransformer(percentage_column)),
    ('extract_time', FunctionTransformer(extract_time))
])

- Apply pipeline

In [629]:
# Concatenate the events data
events = [event_mp01,event_mp02,event_mp03,event_mp04,event_mp05,event_mp06,event_mp07,event_mp08Vrijthof] #mp08stadspark is not used in noise data

In [630]:
# Apply the pipeline
combined_event = pipeline_merge_event.fit_transform(events)
combined_event.head(1000000)

,description,result_timestamp,noise_event_laeq_primary_detected_certainty,noise_event_laeq_primary_detected_class,certainty_percentage,month,day,hour,minute,second,milliseconds
0,MP 01: Naamsestraat 35 Maxim,2022-02-28 08:27:21.737,NaN,NaN,NaN,2,28,8,27,21,737
1,MP 01: Naamsestraat 35 Maxim,2022-02-28 13:58:21.356,NaN,NaN,NaN,2,28,13,58,21,356
2,MP 01: Naamsestraat 35 Maxim,2022-02-28 16:43:15.393,NaN,NaN,NaN,2,28,16,43,15,393
3,MP 01: Naamsestraat 35 Maxim,2022-02-28 19:22:48.428,NaN,NaN,NaN,2,28,19,22,48,428
4,MP 01: Naamsestraat 35 Maxim,2022-02-28 20:32:20.440,NaN,NaN,NaN,2,28,20,32,20,440
...,...,...,...,...,...,...,...,...,...,...,...
81051,MP08bis - Vrijthof,2022-12-29 09:08:11.171,99.0,Human voice - Shouting,99.0%,12,29,9,8,11,171
81052,MP08bis - Vrijthof,2022-12-30 13:54:27.224,99.0,Nature elements - Wind,99.0%,12,30,13,54,27,224
81053,MP08bis - Vrijthof,2022-12-30 13:56:57.225,0.0,Unsupported,0.0%,12,30,13,56,57,225
81054,MP08bis - Vrijthof,2022-12-30 15:09:33.233,100.0,Nature elements - Wind,100.0%,12,30,15,9,33,233


#### Preprocess noise data for merging purpose

In [631]:
# Pipeline on noise data for merging purpose
# Step 1: Convert timestamps to datetime
def convert_to_datetime(df):
    df['result_timestamp'] = pd.to_datetime(df['result_timestamp'], format='%d/%m/%Y %H:%M:%S.%f')
    return df

# Step 2: Extract month, day, hour, minute from timestamps
def extract_time(df):
    df['month'] = df['result_timestamp'].dt.month
    df['day'] = df['result_timestamp'].dt.day
    df['hour'] = df['result_timestamp'].dt.hour
    df['minute'] = df['result_timestamp'].dt.minute
    df['second'] = df['result_timestamp'].dt.second
    df['milliseconds'] = df['result_timestamp'].dt.microsecond // 1000
    return df

# Step 3: Drop columns
def drop_columns(df):
    columns_to_keep = ['description', 'lamax', 'laeq', 'month','day','hour','minute','second','milliseconds']
    columns_to_drop = set(df.columns) - set(columns_to_keep)
    return df.drop(columns=columns_to_drop)

# Step 4: Forward fill missing values
def forward_fill(df):
    return df.ffill()

# Define the pipeline
pipeline_merge_noise = Pipeline([
    ('convert_to_datetime', FunctionTransformer(convert_to_datetime)),
    ('extract_time', FunctionTransformer(extract_time)),
    ('drop_columns', FunctionTransformer(drop_columns)),
    ('forward_fill', FunctionTransformer(forward_fill))
])

#### Apply the pipeline and merge data month by month (to avoid kernel crashing)

In [632]:
merge_columns = ['description','month','day','hour','minute','second','milliseconds']

- Jan

In [633]:
# Concatenate noise data
jan = pd.concat(dfs_jan, ignore_index=True)

In [634]:
# Merge the data
jan = pipeline_merge_noise.fit_transform(jan) # Apply pipeline to noise data
event_jan = combined_event[combined_event['month'] == 1] # Select event data from corresponding month
merged_event_jan = pd.merge(event_jan, jan, on=merge_columns,  how='left') # Merge the event and noise data

In [ ]:
merged_event_jan.head()
#merged_event_jan.to_csv('merged_event_jan.csv', index=False)  

In [635]:
# Delete to reduce memory use
del jan
del event_jan

- Feb

In [636]:
# Concatenate noise data
feb = pd.concat(dfs_feb, ignore_index=True)

In [637]:
# Merge the data
feb = pipeline_merge_noise.fit_transform(feb) # Apply pipeline to noise data
event_feb = combined_event[combined_event['month'] == 2] # Select event data from corresponding month
merged_event_feb = pd.merge(event_feb, feb, on=merge_columns,  how='left') # Merge the event and noise data


In [ ]:
merged_event_feb.head()
#merged_event_feb.to_csv('merged_event_feb.csv', index=False)  

In [638]:
# Delete to reduce memory use
del feb
del event_feb

- Mar

In [639]:
# Concatenate noise data
mar = pd.concat(dfs_mar, ignore_index=True)

In [640]:
# Merge the data
mar_ = pipeline_merge_noise.fit_transform(mar) # Apply pipeline to noise data
event_mar = combined_event[combined_event['month'] == 3] # Select event data from corresponding month
merged_event_mar = pd.merge(event_mar, mar_, on=merge_columns,  how='left') # Merge the event and noise data

In [23]:
merged_event_mar.head()
#merged_event_mar.to_csv('merged_event_mar.csv', index=False)  

,description,result_timestamp,noise_event_laeq_primary_detected_certainty,noise_event_laeq_primary_detected_class,certainty_percentage,month,day,hour,minute,second,milliseconds,lamax,laeq
0,MP 01: Naamsestraat 35 Maxim,2022-03-01 00:07:59.463,NaN,NaN,NaN,3,1,0,7,59,463,74.1,70.8
1,MP 01: Naamsestraat 35 Maxim,2022-03-01 01:24:17.470,NaN,NaN,NaN,3,1,1,24,17,470,82.1,78.7
2,MP 01: Naamsestraat 35 Maxim,2022-03-01 01:33:16.470,NaN,NaN,NaN,3,1,1,33,16,470,82.4,78.8
3,MP 01: Naamsestraat 35 Maxim,2022-03-01 02:31:01.476,NaN,NaN,NaN,3,1,2,31,1,476,NaN,NaN
4,MP 01: Naamsestraat 35 Maxim,2022-03-01 03:45:02.482,NaN,NaN,NaN,3,1,3,45,2,482,74.8,72.6


In [641]:
# Delete to reduce memory use
del mar
del event_mar

- Apr

In [642]:
# Concatenate noise data
apr = pd.concat(dfs_apr, ignore_index=True)

In [643]:
# Merge the data
apr_ = pipeline_merge_noise.fit_transform(apr) # Apply pipeline to noise data
event_apr = combined_event[combined_event['month'] == 4] # Select event data from corresponding month
merged_event_apr = pd.merge(event_apr, apr_, on=merge_columns,  how='left') # Merge the event and noise data

In [ ]:
merged_event_apr.head()
#merged_event_apr.to_csv('merged_event_apr.csv', index=False)  

In [644]:
# Delete to reduce memory use
del apr
del event_apr

- May

In [645]:
# Concatenate noise data
may = pd.concat(dfs_may, ignore_index=True)

In [646]:
# Merge the data
may_ = pipeline_merge_noise.fit_transform(may) # Apply pipeline to noise data
event_may = combined_event[combined_event['month'] == 5] # Select event data from corresponding month
merged_event_may = pd.merge(event_may, may_, on=merge_columns,  how='left') # Merge the event and noise data

In [ ]:
merged_event_may.head()
#merged_event_may.to_csv('merged_event_may.csv', index=False)  

In [647]:
# Delete to reduce memory use
del may
del event_may

- Jun

In [648]:
# Concatenate noise data
jun = pd.concat(dfs_jun, ignore_index=True)

In [649]:
# Merge the data
jun_ = pipeline_merge_noise.fit_transform(jun)
event_jun = combined_event[combined_event['month'] == 6]
merged_event_jun = pd.merge(event_jun, jun_, on=merge_columns,  how='left')

In [ ]:
merged_event_jun.head()
#merged_event_jun.to_csv('merged_event_jun.csv', index=False)  

In [650]:
# Delete to reduce memory use
del jun
del event_jun

- Jul

In [651]:
# Concatenate noise data
jul = pd.concat(dfs_jul, ignore_index=True)

In [652]:
# Merge the data
jul_ = pipeline_merge_noise.fit_transform(jul)
event_jul = combined_event[combined_event['month'] == 7]
merged_event_jul = pd.merge(event_jul, jul_, on=merge_columns,  how='left')

In [ ]:
merged_event_jul.head()
#merged_event_jul.to_csv('merged_event_jul.csv', index=False)

In [653]:
# Delete to reduce memory use
del jul
del event_jul

- Aug

In [654]:
# Concatenate noise data
aug = pd.concat(dfs_aug, ignore_index=True)

In [655]:
# Merge the data
aug_ = pipeline_merge_noise.fit_transform(aug)
event_aug = combined_event[combined_event['month'] == 8]
merged_event_aug = pd.merge(event_aug, aug_, on=merge_columns,  how='left')

In [ ]:
merged_event_aug.head()
#merged_event_aug.to_csv('merged_event_aug.csv', index=False)  

In [656]:
# Delete to reduce memory use
del aug
del event_aug

- Sep

In [657]:
# Concatenate noise data
sep = pd.concat(dfs_sep, ignore_index=True)

In [658]:
# Merge the data
sep_ = pipeline_merge_noise.fit_transform(sep)
event_sep = combined_event[combined_event['month'] == 9]
merged_event_sep = pd.merge(event_sep, sep_, on=merge_columns,  how='left')

In [ ]:
merged_event_sep.head()
#merged_event_sep.to_csv('merged_event_sep.csv', index=False)  

In [659]:
# Delete to reduce memory use
del sep
del event_sep

- Oct

In [660]:
# Concatenate noise data
oct = pd.concat(dfs_oct, ignore_index=True)

In [661]:
# Merge the data
oct_ = pipeline_merge_noise.fit_transform(oct)
event_oct = combined_event[combined_event['month'] == 10]
merged_event_oct = pd.merge(event_oct, oct_, on=merge_columns,  how='left')

In [ ]:
merged_event_oct.head()
#merged_event_oct.to_csv('merged_event_oct.csv', index=False)  

In [662]:
# Delete to reduce memory use
del oct
del event_oct

- Nov

In [663]:
# Concatenate noise data
nov = pd.concat(dfs_nov, ignore_index=True)

In [664]:
# Merge the data
nov_ = pipeline_merge_noise.fit_transform(nov)
event_nov = combined_event[combined_event['month'] == 11]
merged_event_nov = pd.merge(event_nov, nov_, on=merge_columns,  how='left')

In [ ]:
merged_event_nov.head()
#merged_event_nov.to_csv('merged_event_nov.csv', index=False)  

In [665]:
# Delete to reduce memory use
del nov
del event_nov

- Dec

In [666]:
# Concatenate noise data
dec = pd.concat(dfs_dec, ignore_index=True)

In [667]:
# Merge the data
dec_ = pipeline_merge_noise.fit_transform(dec)
event_dec = combined_event[combined_event['month'] == 12]
merged_event_dec = pd.merge(event_dec, dec_, on=merge_columns,  how='left')

In [ ]:
merged_event_dec.head()
#merged_event_dec.to_csv('merged_event_dec.csv', index=False)  

In [668]:
# Delete to reduce memory use
del dec
del event_dec

##### Concatenate the datasets

In [669]:
# Concatenate event data
dfs = [merged_event_jan,merged_event_feb,merged_event_mar,merged_event_apr,merged_event_may,merged_event_jun,merged_event_jul,merged_event_aug,merged_event_sep,merged_event_oct,merged_event_nov,merged_event_dec]
combined_noise_event = pd.concat(dfs, ignore_index=True)

In [670]:
combined_noise_event.head()
# combined_noise_event.to_csv('combined_noise_event.csv', index=False)  

,description,result_timestamp,noise_event_laeq_primary_detected_certainty,noise_event_laeq_primary_detected_class,certainty_percentage,month,day,hour,minute,second,milliseconds,lamax,laeq
0,MP 03: Naamsestraat 62 Taste,2022-01-01 00:01:31.362,NaN,NaN,NaN,1,1,0,1,31,362,70.4,68.8
1,MP 03: Naamsestraat 62 Taste,2022-01-01 00:02:19.368,NaN,NaN,NaN,1,1,0,2,19,368,NaN,NaN
2,MP 03: Naamsestraat 62 Taste,2022-01-01 00:02:23.368,NaN,NaN,NaN,1,1,0,2,23,368,NaN,NaN
3,MP 03: Naamsestraat 62 Taste,2022-01-01 00:02:33.370,NaN,NaN,NaN,1,1,0,2,33,370,93.4,87.4
4,MP 03: Naamsestraat 62 Taste,2022-01-01 00:02:57.373,NaN,NaN,NaN,1,1,0,2,57,373,94.3,87.0
